<a href="https://colab.research.google.com/github/DhruvAgg10/Python-projects/blob/main/batteryrul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Hawaii Natural Energy Institute examined 14 NMC-LCO 18650 batteries with a nominal capacity of 2.8 Ah, which were cycled over 1000 times at 25°C with a CC-CV charge rate of C/2 rate and discharge rate of 1.5C.

From that source dataset, I created features that showcase the voltage and current behaviour over each cycle. Those features can be used to predict the remaining useful life (RUL) of the batteries. The dataset contains the summary of the 14 batteries.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#import xgb regressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [ ]:
# Load dataset
file_path = '/content/Battery_RUL.csv'
df = pd.read_csv(file_path, header=None)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8
0,Cycle_Index,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s),RUL
1,1.0,2595.3,1151.4885,3.67,3.211,5460.001,6755.01,10777.82,1112
2,2.0,7408.64,1172.5125000000007,4.246,3.22,5508.991999999998,6762.02,10500.35,1111
3,3.0,7393.76,1112.991999999991,4.249,3.224,5508.993000000002,6762.02,10420.38,1110
4,4.0,7385.5,1080.3206666666665,4.25,3.225,5502.015999999996,6762.02,10322.81,1109


In [ ]:
# Set the first row as column headers and drop it
df.columns =['Cycle_Index', 'Discharge time(s)', 'Decrement 3.6-3.4V(s)', 'max voltage discharge(V)','Min. Voltage Charg. (V)',	'Time at 4.15V (s)',	'Time constant current (s)',	'Charging time (s)	','RUL']



In [ ]:
#drop row 0
df = df.drop(0)
df

,Cycle_Index,Discharge time(s),Decrement 3.6-3.4V(s),max voltage discharge(V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s)\t,RUL
1,1.0,2595.3,1151.4885,3.67,3.211,5460.001,6755.01,10777.82,1112
2,2.0,7408.64,1172.5125000000007,4.246,3.22,5508.991999999998,6762.02,10500.35,1111
3,3.0,7393.76,1112.991999999991,4.249,3.224,5508.993000000002,6762.02,10420.38,1110
4,4.0,7385.5,1080.3206666666665,4.25,3.225,5502.015999999996,6762.02,10322.81,1109
5,6.0,65022.75,29813.487,4.29,3.398,5480.992000000013,53213.54,56699.65,1107
...,...,...,...,...,...,...,...,...,...
15060,1108.0,770.44,179.52380952239037,3.773,3.742,922.7750000003724,1412.38,6678.88,4
15061,1109.0,771.12,179.52380952239037,3.773,3.744,915.5120000001044,1412.31,6670.38,3
15062,1110.0,769.12,179.35714285634458,3.773,3.742,915.5129999984056,1412.31,6637.12,2
15063,1111.0,773.88,162.37466666661203,3.763,3.839,539.375,1148.0,7660.62,1


In [ ]:
df.isnull()

,Cycle_Index,Discharge time(s),Decrement 3.6-3.4V(s),max voltage discharge(V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s)\t,RUL
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
15060,False,False,False,False,False,False,False,False,False
15061,False,False,False,False,False,False,False,False,False
15062,False,False,False,False,False,False,False,False,False
15063,False,False,False,False,False,False,False,False,False


In [ ]:
#seperate rul
X= df.drop('RUL', axis=1)
y= df['RUL']

In [ ]:
X

,Cycle_Index,Discharge time(s),Decrement 3.6-3.4V(s),max voltage discharge(V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s)\t
1,1.0,2595.3,1151.4885,3.67,3.211,5460.001,6755.01,10777.82
2,2.0,7408.64,1172.5125000000007,4.246,3.22,5508.991999999998,6762.02,10500.35
3,3.0,7393.76,1112.991999999991,4.249,3.224,5508.993000000002,6762.02,10420.38
4,4.0,7385.5,1080.3206666666665,4.25,3.225,5502.015999999996,6762.02,10322.81
5,6.0,65022.75,29813.487,4.29,3.398,5480.992000000013,53213.54,56699.65
...,...,...,...,...,...,...,...,...
15060,1108.0,770.44,179.52380952239037,3.773,3.742,922.7750000003724,1412.38,6678.88
15061,1109.0,771.12,179.52380952239037,3.773,3.744,915.5120000001044,1412.31,6670.38
15062,1110.0,769.12,179.35714285634458,3.773,3.742,915.5129999984056,1412.31,6637.12
15063,1111.0,773.88,162.37466666661203,3.763,3.839,539.375,1148.0,7660.62


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))



In [ ]:
# Train XGBoost model
xgb_model = XGBRegressor(n_estimators=100, random_state=42)

# Convert all columns of X_train to numeric, errors='coerce' will replace invalid values with NaN
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

# Now you can fit the model
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
xgb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))

In [ ]:
# Print RMSE for evaluation
print(f'Random Forest RMSE: {rf_rmse}')
print(f'XGBoost RMSE: {xgb_rmse}')

Random Forest RMSE: 3.787344184358378
XGBoost RMSE: 4.136041832599814


In [ ]:
# Function to predict RUL
def predict_rul(input_features):
    input_array = np.array(input_features).reshape(1, -1)
    prediction = rf_model.predict(input_array)[0]
    return prediction

# Example usage
sample_input = [10,5945.44,1216.921,4.014, 3.501, 5009.994, 5954.91, 5954.91]
predicted_rul = predict_rul(sample_input)
print(f'Predicted RUL for sample input: {predicted_rul}')

Predicted RUL for sample input: 1101.35


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
# Save the model
joblib.dump(rf_model, "battery_rul_model.pkl")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
from google.colab import files

# Download the saved model file
files.download("battery_rul_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>